Fick lär mig något nytt precis:

fick bekräftat av Richard att när man gör cross_validation eller cross_val_score så anropas fit() automatisk i den funktionen för varje split. Dvs. man behöver alltså inte använda fit() separat innan cross val()

## Task 1: Working with a dataset with categorical features

In Assignment 1, we didn't have to do much preprocessing, because all the features in the two datasets were numerical. (Actually, in the second dataset, we removed all non-numerical features.) In this assignment, we'll instead consider how to deal with non-numerical features.

We'll use the famous Adult dataset. This is a binary classification task, where our task is to predict whether an American individual earns more than $50,000 a year, given a number of numerical and categorical features. (The dataset was extracted from a 1994 census database.)

### Step 1. Reading the data

Please download the two CSV files, the training set and the test set, and save them into your working directory This is the official train/test split defined by the people who created the dataset. It's the same data as in the the public distribution, except that we converted the format into a standard CSV format.

1. Write code to read the CSV file, for instance by using Pandas as in Assignment 1. 
2. Then split the data into an input part X and an output part Y. The output variable, which the classifier will predict, is called target.





In [29]:
# import required libs
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer

In [2]:
# Import csv file for Training data
train = pd.read_csv('adult_train.csv')

# Import csv file for Test data
test = pd.read_csv('adult_test.csv')

# Shuffle the dataset.
train_shuffled = train.sample(frac=1.0, random_state=0)
test_shuffled = test.sample(frac=1.0, random_state=0)

# Split into input part X and output part Y.
Xtrain = train_shuffled.drop('target', axis=1)
Ytrain = train_shuffled['target']
Xtest = test_shuffled.drop('target', axis=1)
Ytest = test_shuffled['target']

In [3]:
Xtrain.head(3)

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
22278,49,Local-gov,HS-grad,9,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States
8950,49,Private,HS-grad,9,Divorced,Other-service,Not-in-family,Black,Female,0,0,40,United-States
7838,31,Private,Prof-school,15,Never-married,Prof-specialty,Not-in-family,White,Male,0,0,50,United-States


### Step 2: Encoding the features as numbers.

If you look at the data, you will note that it contains several features with categorical values, such as workclass, education etc. All scikit-learn models work with numerical data internally; this means that the categorical features need to be converted to numbers. The most straightforward way to carry out such a conversion is to use one-hot encoding of the features, also known as dummy variables in statistics. In this approach, we define one new column for each observed value of the feature.

Scikit-learn includes a number of tools that can do one-hot encoding of categorical features and we'll see how to use one of them, the DictVectorizer. An alternative approach that is a bit more Pandas-friendly and gives more low-level control is to use the recently introduced ColumnTransformer; if you're interested, you can read an introduction to this approach here. We won't use a ColumnTransformer here because it will make Task 3 in this assignment a bit too annoying to solve.

The DictVectorizer is used when we store our features as named attributes in dictionaries. For instance, we could represent one individual from the Adult dataset as follows:

    {'age': 44,
    'workclass': 'Private',
    'education': 'Some-college',
    'education-num': 10,
    'marital-status': 'Married-civ-spouse',
    'occupation': 'Machine-op-inspct',
    'relationship': 'Husband',
    'race': 'Black',
    'sex': 'Male',
    'capital-gain': 7688,
    'capital-loss': 0,
    'hours-per-week': 40,
    'native-country': 'United-States'}

Pandas includes a utility to convert a DataFrame into a list of dictionaries:

    dicts_for_my_training_data = my_training_data.to_dict('records')

Then make a DictVectorizer and apply it, writing something like the following:

    dv = DictVectorizer()
    X_train_encoded = dv.fit_transform(dicts_for_my_training_data)

The method fit_transform will first call fit, which as usual is the "training" method. For a DictVectorizer, "training" consists of building the mapping from categories to column positions. Then, the transform method will be called, which converts the data into a matrix.

Now that you have a numerical representation of the data, you can compute a cross-validation accuracy for the training set using one of the classifiers you explored in Programming Assignment 1.

To handle the test data, you just call transform, because this time the vectorizer does not need to be "trained." Use this to compute the accuracy on the test set.

    X_test_encoded = dv.transform(dicts_for_my_test_data)

In [4]:
# convert training data to dict - creates a list of dicts 
# where each row's (person's) information is gathered in one dict, 
dicts_train = Xtrain.to_dict('records')

# show first 2 person's informations
dicts_train[:2] 

[{'age': 49,
  'workclass': 'Local-gov',
  'education': 'HS-grad',
  'education-num': 9,
  'marital-status': 'Married-civ-spouse',
  'occupation': 'Transport-moving',
  'relationship': 'Husband',
  'race': 'White',
  'sex': 'Male',
  'capital-gain': 0,
  'capital-loss': 0,
  'hours-per-week': 40,
  'native-country': 'United-States'},
 {'age': 49,
  'workclass': 'Private',
  'education': 'HS-grad',
  'education-num': 9,
  'marital-status': 'Divorced',
  'occupation': 'Other-service',
  'relationship': 'Not-in-family',
  'race': 'Black',
  'sex': 'Female',
  'capital-gain': 0,
  'capital-loss': 0,
  'hours-per-week': 40,
  'native-country': 'United-States'}]

Compute a cross-validation accuracy for the training set using one of the classifiers we explored in Programming Assignment 1.

To handle the test data, you just call transform, because this time the vectorizer does not need to be "trained." Use this to compute the accuracy on the test set.

In [41]:
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier

In [44]:
dummy = DummyClassifier(strategy='most_frequent')
dummyScores = cross_val_score(dummy, Xtrain, Ytrain)
dummy.fit(Xtrain,Ytrain)
dummy_score = np.mean(dummyScores)

print("Average score:",dummy_score)

Average score: 0.7591904454179904


In [5]:
# create d DictVectorizer 
dv = DictVectorizer()

# one-hot encode the list with all dicts
Xtrain_encoded = dv.fit_transform(dicts_train)

In [64]:
accuracy_score(Ytest, dummy.predict(Xtest))

0.7637737239727289

In [18]:
data = Xtrain

By shoosing a top amount of data, we can see the remainding of it as noice data.

In [27]:
# Import label encoder 
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

In [44]:
# label to number
data['workclass'] = label_encoder.fit_transform(data['workclass'])
data['education'] = label_encoder.fit_transform(data['education'])
data['marital-status'] = label_encoder.fit_transform(data['marital-status'])
data['occupation'] = label_encoder.fit_transform(data['occupation'])
data['relationship'] = label_encoder.fit_transform(data['relationship'])
data['race'] = label_encoder.fit_transform(data['race'])
data['native-country'] = label_encoder.fit_transform(data['native-country'])
data['sex'] = label_encoder.fit_transform(data['sex'])
print(xx.head())

       age  workclass  education  education-num  marital-status  occupation  \
22278   49          2         11              9               2          14   
8950    49          4         11              9               0           8   
7838    31          4         14             15               4          10   
16505   53          4         15             10               0           1   
19140   30          4          9             13               2          10   

       relationship  race  sex  capital-gain  capital-loss  hours-per-week  \
22278             0     4    1             0             0              40   
8950              1     2    0             0             0              40   
7838              1     4    1             0             0              50   
16505             1     4    0             0             0              43   
19140             0     4    1             0             0              40   

       native-country  
22278              39  
8950    

In [6]:
# fortsätt här
dicts_test = Xtest.to_dict('records')
dicts_test[:2]

[{'age': 44,
  'workclass': 'Private',
  'education': 'Some-college',
  'education-num': 10,
  'marital-status': 'Married-civ-spouse',
  'occupation': 'Transport-moving',
  'relationship': 'Husband',
  'race': 'White',
  'sex': 'Male',
  'capital-gain': 0,
  'capital-loss': 0,
  'hours-per-week': 48,
  'native-country': 'United-States'},
 {'age': 37,
  'workclass': 'Private',
  'education': 'HS-grad',
  'education-num': 9,
  'marital-status': 'Divorced',
  'occupation': 'Exec-managerial',
  'relationship': 'Unmarried',
  'race': 'White',
  'sex': 'Female',
  'capital-gain': 0,
  'capital-loss': 0,
  'hours-per-week': 50,
  'native-country': 'United-States'}]

In [7]:
X_test_encoded = dv.transform(dicts_test)

In [8]:
X_test_encoded

<16281x107 sparse matrix of type '<class 'numpy.float64'>'
	with 211653 stored elements in Compressed Sparse Row format>

### Step 3. Combining the steps.

In the example above, we first transformed the list of dictionaries into a numerical matrix, and then we used this matrix when training the classifier. A separate preprocessing step was carried out for the test set.

In machine learning setups, we often use long chains of preprocessing steps. The one-hot encoding is one example, and other such steps might be scaling, feature selection, imputation of missing values, etc. As you can imagine, keeping track of the preprocessing steps can be tedious and error-prone, so it makes sense to handle such preprocessing chains automatically.

A Pipeline consists of a sequence of scikit-learn modules. The most convenient way to build a Pipeline is to use the utility function make_pipeline. For instance, to build a pipeline consisting of a vectorization step and then a decision tree classifier, we could write

    from sklearn.pipeline import make_pipeline
    
    pipeline = make_pipeline(
    DictVectorizer(),
    DecisionTreeClassifier() )
    
The Pipeline can be treated as any classifier: we can call fit and predict as usual. Concretely, when we call fit on a Pipeline, it will in turn call fit_transform on all intermediate steps and then fit on the final step. When we call predict, transform will be called on the intermediate steps and then predict on the final step.

Build a pipeline that includes the classifier that you selected previously, and make sure that it works.

In [5]:
import pandas as pd

train_data = pd.read_csv('adult_train.csv')

n_cols = len(train_data.columns)
Xtrain = train_data.iloc[:, :n_cols-1].to_dict('records')
Ytrain = train_data.iloc[:, n_cols-1]

test_data = pd.read_csv('adult_test.csv')
Xtest = test_data.iloc[:, :n_cols-1].to_dict('records')
Ytest = test_data.iloc[:, n_cols-1]

Create pipeline

In [78]:
# from example
import warnings
warnings.filterwarnings('ignore')

import numpy as np

from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score

from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest

preprocessing_pipeline = make_pipeline(DictVectorizer(), StandardScaler(with_mean=False),SelectKBest(k=100),)

preprocessing_pipeline.fit(Xtrain, Ytrain)
X_vec = preprocessing_pipeline.transform(Xtrain)


In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(solver='liblinear')

#param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
from sklearn.utils.fixes import loguniform
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2', 'elasticnet', 'none']}

gridsearch = GridSearchCV(clf, param_grid)

gridsearch.fit(X_vec, Ytrain);

In [27]:
gridsearch.best_params_

{'C': 0.1, 'penalty': 'l2'}

In [25]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon

C_distr = expon(scale=2)
param_grid_random = {'C': C_distr, 'penalty': ['l1', 'l2']}
randomsearch = RandomizedSearchCV(clf, param_grid_random, n_iter=50)
randomsearch.fit(X_vec, Ytrain);

KeyboardInterrupt: 

In [26]:
randomsearch.best_params_

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

In [63]:
# kod exemple
best_C = randomsearch.best_params_['C']
best_penalty = randomsearch.best_params_['penalty']

pipeline = make_pipeline(
    DictVectorizer(),
    StandardScaler(with_mean=False),
    SelectKBest(k=100),
    LogisticRegression(C=best_C, penalty=best_penalty)
)

pipeline.fit(Xtrain, Ytrain)
accuracy_score(Ytest, pipeline.predict(Xtest))

0.8522203795835637

In [15]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn import tree

prep_pipeline = make_pipeline(DictVectorizer(), StandardScaler(with_mean=False),SelectKBest(k=100),)

prep_pipeline.fit(Xtrain, Ytrain)
X_vec = prep_pipeline.transform(Xtrain)

In [17]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear')
param_grid = {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2']}
gridsearch = GridSearchCV(lr, param_grid)
gridsearch.fit(X_vec, Ytrain);

In [29]:
from sklearn import decomposition, datasets
from sklearn import tree
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# parameteers for pipeline
sc = StandardScaler()
pca = decomposition.PCA()
dtreeCLF = tree.DecisionTreeClassifier()   

pipe = Pipeline(steps=[("sc", sc),
                           ("pca", pca),
                           ("dtreeCLF", dtreeCLF)])

#pipe.fit(Xtrain, Ytrain)

## WORKS

source: https://www.cse.chalmers.se/~richajo/dit866/lectures/l3/AutomaticHyperparameterTuning.html

In [1]:
import pandas as pd

train_data = pd.read_csv('adult_train.csv')

n_cols = len(train_data.columns)
Xtrain = train_data.iloc[:, :n_cols-1].to_dict('records')
Ytrain = train_data.iloc[:, n_cols-1]

test_data = pd.read_csv('adult_test.csv')
Xtest = test_data.iloc[:, :n_cols-1].to_dict('records')
Ytest = test_data.iloc[:, n_cols-1]

In [2]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [3]:
preprocessing_pipeline = make_pipeline(
    DictVectorizer(),
    StandardScaler(with_mean=False),
    SelectKBest(k=100),
)

In [4]:
preprocessing_pipeline.fit(Xtrain, Ytrain)
X_vec = preprocessing_pipeline.transform(Xtrain)
dtc = DecisionTreeClassifier()

In [5]:
param_grid = {'criterion':['gini', 'entropy'], 
                     'splitter':['best', 'random'],
                     'max_features':['auto', 'sqrt', 'log2']}

#'max_depth':[1,2,3,4,5,6,7,8,9,10],
#'min_samples_leaf': [1,2,3,4,5],

In [6]:
gridsearch = GridSearchCV(dtc, param_grid)
gridsearch.fit(X_vec, Ytrain);

In [7]:
gridsearch.best_params_

{'criterion': 'gini', 'max_features': 'sqrt', 'splitter': 'best'}

In [8]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon

In [9]:
C_distr = expon(scale=2)

In [9]:
param_grid_random = {'criterion':['gini', 'entropy'], 
                     'splitter':['best', 'random'],
                     'max_features':['auto', 'sqrt', 'log2']}

In [10]:
randomsearch = RandomizedSearchCV(dtc, param_grid_random, n_iter=10)
randomsearch.fit(X_vec, Ytrain);

In [11]:
randomsearch.best_params_

{'splitter': 'best', 'max_features': 'auto', 'criterion': 'entropy'}

In [12]:
b_splitter = randomsearch.best_params_['splitter']
b_max_features = randomsearch.best_params_['max_features']
b_criterion = randomsearch.best_params_['criterion']
#b_min_samples_leaf = randomsearch.best_params_['min_samples_leaf']
#b_max_depth = randomsearch.best_params_['max_depth']

In [13]:
print(b_splitter, b_max_features, b_criterion)

best auto entropy


In [14]:
pipeline = make_pipeline(
    DictVectorizer(),
    StandardScaler(with_mean=False),
    SelectKBest(k=100),
    DecisionTreeClassifier()
)

In [38]:
pipeline.fit(Xtrain, Ytrain)

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('standardscaler', StandardScaler(with_mean=False)),
                ('selectkbest', SelectKBest(k=100)),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [39]:
accuracy_score(Ytest, pipeline.predict(Xtest))

0.8189914624408821

## *OLD BELOW*

In [46]:
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

In [47]:
pipeline = make_pipeline(DictVectorizer(), DecisionTreeClassifier())

In [48]:
pipeline

Pipeline(steps=[('dictvectorizer', DictVectorizer()),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [49]:
pipeline.fit(Xtrain, Ytrain)

AttributeError: 'str' object has no attribute 'items'

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a6ba0eaf-97fa-4fd0-ab2f-858132e990c1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>